In [1]:

import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50


import os

In [7]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join("F:/yasmin/roi/","model_roi_save_s2.h5"))
print(model2.summary())

model = models.Sequential()
for layer in model2.layers[:-1]: # go through until last layer
    model.add(layer)
    
model.add(layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(2, activation='softmax'))
print(model.summary())


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5125      
Total params: 126,354,309
Trainable params: 118,863,365
Non-trainable params: 7,490,944
__________________________________

In [13]:
for layer in model2.layers[:]:
   layer.trainable = False

print('model2 is now NOT trainable')

model is now NOT trainable


In [9]:
for i, layer in enumerate(model2.layers):
   print(i, layer.name, layer.trainable)
print(model.summary())

0 resnet50 False
1 flatten False
2 dropout False
3 dense False
4 dropout_1 False
5 dense_1 False
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_______________________

In [10]:
train_dir ='F:/yasmin/whole image 2classes/train'
test_dir = 'F:/yasmin/whole image 2classes/test'

In [11]:
train_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input,
    rotation_range=25.,
    channel_shift_range=20.,
    validation_split=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
        )

test_datagen = ImageDataGenerator(
    preprocessing_function= \
    keras.applications.resnet50.preprocess_input
    )


In [12]:

train_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),classes=['benign','malignant'], batch_size=64,color_mode='rgb',subset='training')
validation_batches = train_datagen.flow_from_directory(train_dir, target_size=(224,224),classes=['benign','malignant'], batch_size=64,color_mode='rgb',subset='validation')
test_batches = test_datagen.flow_from_directory(test_dir, target_size=(224,224), classes=['benign','malignant'],color_mode='rgb', batch_size=1)


Found 1776 images belonging to 2 classes.
Found 442 images belonging to 2 classes.
Found 474 images belonging to 2 classes.


In [13]:
from tensorflow.keras.optimizers import Adam


model.compile(optimizer=Adam(lr=.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [14]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/whole image 2classes/new_model_check2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [15]:
history = model.fit(train_batches, steps_per_epoch = 1776 // 64, 
   validation_data=validation_batches, validation_steps = 442 // 64, epochs=30, callbacks=callbacks_list, verbose=1)



Epoch 1/30
27/27 [==============================] - ETA: 0s - loss: 2.9983 - accuracy: 0.5129
Epoch 00001: val_accuracy improved from -inf to 0.57292, saving model to F:/yasmin/whole image 2classes/new_model_check2.h5
27/27 [==============================] - 326s 12s/step - loss: 2.9983 - accuracy: 0.5129 - val_loss: 2.8215 - val_accuracy: 0.5729
Epoch 2/30
27/27 [==============================] - ETA: 0s - loss: 2.8208 - accuracy: 0.5234
Epoch 00002: val_accuracy did not improve from 0.57292
27/27 [==============================] - 279s 10s/step - loss: 2.8208 - accuracy: 0.5234 - val_loss: 2.7314 - val_accuracy: 0.4818
Epoch 3/30
27/27 [==============================] - ETA: 0s - loss: 2.6697 - accuracy: 0.5426
Epoch 00003: val_accuracy improved from 0.57292 to 0.58333, saving model to F:/yasmin/whole image 2classes/new_model_check2.h5
27/27 [==============================] - 265s 10s/step - loss: 2.6697 - accuracy: 0.5426 - val_loss: 2.5831 - val_accuracy: 0.5833
Epoch 4/30
27/27 [=

Epoch 30/30
27/27 [==============================] - ETA: 0s - loss: 1.7178 - accuracy: 0.5415
Epoch 00030: val_accuracy improved from 0.58333 to 0.58594, saving model to F:/yasmin/whole image 2classes/new_model_check2.h5
27/27 [==============================] - 260s 10s/step - loss: 1.7178 - accuracy: 0.5415 - val_loss: 1.6962 - val_accuracy: 0.5859


In [16]:
model.save('F:/yasmin/whole image 2classes/new_model_save_s2.h5')

In [3]:
for layer in model2.layers[:]:
   layer.trainable = True

print('Last block of the model2 is now trainable')

Last block of the conv_base is now trainable


In [4]:
for i, layer in enumerate(model2.layers):
   print(i, layer.name, layer.trainable)

0 resnet50 False
1 flatten False
2 dropout False
3 dense False
4 dropout_1 False
5 dense_2 False
6 dense_3 False
7 dense_4 False


In [19]:
model.compile(optimizer=Adam(lr=.00001), loss='binary_crossentropy', metrics=['accuracy'])

print("model compiled")
print(model.summary())

model compiled
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dropout (Dropout)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 102

In [20]:
from tensorflow.keras.callbacks import  ModelCheckpoint


filepath = "F:/yasmin/whole image 2classes/new_model_checkfinal.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_batches, steps_per_epoch = 1776 // 64, 
                    validation_data=validation_batches, validation_steps = 442 // 64, epochs=20, callbacks=callbacks_list, verbose=1)



Epoch 1/20
 1/27 [>.............................] - ETA: 0s - loss: 1.7849 - accuracy: 0.4844

In [33]:
model.save('F:/yasmin/whole image 2classes/new_model_save_final.h5')

In [ ]:
model.load_weights('F:/yasmin/whole image 2classes/new_model_check2.h5')
test_loss, test_acc = model.evaluate_generator(test_batches, steps= 3561 // 64, verbose=1)
print('test acc:', test_acc)